In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime
import sqlite3

# Carga de datos

Elegimos el datasest "steam-games.csv", que contiene todos los datos de todos los juegos que hay en la plataforma de distribución digital de videojuegos Steam hasta mayo de 2024 inclusive.

In [2]:
data = pd.read_csv("steam-games.csv")
data.head(5)


,app_id,title,release_date,genres,categories,developer,publisher,original_price,discount_percentage,discounted_price,...,win_support,mac_support,linux_support,awards,overall_review,overall_review_%,overall_review_count,recent_review,recent_review_%,recent_review_count
0,730,Counter-Strike 2,"21 Aug, 2012","Action, Free to Play","Cross-Platform Multiplayer, Steam Trading Card...",Valve,Valve,NaN,NaN,Free,...,True,False,True,1,Very Positive,87.0,8062218.0,Mostly Positive,79.0,57466.0
1,570,Dota 2,"9 Jul, 2013","Action, Strategy, Free to Play","Steam Trading Cards, Steam Workshop, SteamVR C...",Valve,Valve,NaN,NaN,Free,...,True,True,True,0,Very Positive,81.0,2243112.0,Mostly Positive,72.0,23395.0
2,2215430,Ghost of Tsushima DIRECTOR'S CUT,"16 May, 2024","Action, Adventure","Single-player, Online Co-op, Steam Achievement...",Sucker Punch Productions,PlayStation PC LLC,NaN,NaN,"₹3,999.00",...,True,False,False,0,Very Positive,89.0,12294.0,NaN,NaN,NaN
3,1245620,ELDEN RING,"24 Feb, 2022","Action, RPG","Single-player, Online PvP, Online Co-op, Steam...",FromSoftware Inc.,FromSoftware Inc.,NaN,NaN,"₹3,599.00",...,True,False,False,6,Very Positive,93.0,605191.0,Very Positive,94.0,7837.0
4,1085660,Destiny 2,"1 Oct, 2019","Action, Adventure, Free to Play","Single-player, Online PvP, Online Co-op, Steam...",Bungie,Bungie,NaN,NaN,Free,...,True,False,False,0,Very Positive,80.0,594713.0,Mostly Positive,73.0,4845.0


# Análisis de preprocesamiento

En este paso analizamos los datos, sus tipos y la cantidad de nulos para conocer la estructura de los datos y luego ser procesados en el siguiente paso.

In [3]:
data.shape

(42497, 24)

In [4]:
data.isnull().sum()

app_id                      0
title                       0
release_date               57
genres                     87
categories                 45
developer                 190
publisher                 211
original_price          37638
discount_percentage     37638
discounted_price          240
dlc_available               0
age_rating                  0
content_descriptor      40122
about_description         138
win_support                 0
mac_support                 0
linux_support               0
awards                      0
overall_review           2477
overall_review_%         2477
overall_review_count     2477
recent_review           36994
recent_review_%         36994
recent_review_count     36994
dtype: int64

In [5]:
data.dtypes

app_id                    int64
title                    object
release_date             object
genres                   object
categories               object
developer                object
publisher                object
original_price           object
discount_percentage      object
discounted_price         object
dlc_available             int64
age_rating                int64
content_descriptor       object
about_description        object
win_support                bool
mac_support                bool
linux_support              bool
awards                    int64
overall_review           object
overall_review_%        float64
overall_review_count    float64
recent_review            object
recent_review_%         float64
recent_review_count     float64
dtype: object

In [6]:
duplicate_rows = data[data.duplicated(subset=['title', 'publisher', 'developer'])]
print(len(duplicate_rows))

0


Podemos observar que se trata de un set de datos que cuenta con 42.497 datos para 24 columnas. Cuenta con varias columnas con datos nulos y distintos tipos de datos (enteros, booleanos, cadenas, flotantes). No tiene juegos duplicados (si tiene titulos repetidos pero al no ser del mismo 'publisher' ni 'developer' consideramos que se trata de juegos distintos).

# Preprocesamiento de los datos

Creamos una copia del dataset original para que no sea alterado y trabajaremos sobre la copia

In [7]:
data_cpy = data.copy()

## Imputacion de Nulls

En esta sección se busca resolver los valores nulos del dataset. Hay diferentes formas de resolverlo, y las irémos aplicando a continuación.

---



Empezando por la columna "release_date", como son pocos los nulos buscamos en Steam o Google y los completamos

In [8]:
cant_nulos_release_date = data_cpy.isnull().sum()["release_date"]
print ("La cantidad de datos nulos en 'release_date' son: " + str(cant_nulos_release_date))

arr_release_date = np.array(['27 Oct, 2010', '13 Nov, 2009', '18 Feb, 2009', '20 Oct, 2011', '1 Nov, 2015', '25 Feb, 2010', '1 Sep, 2015', '8 Oct, 2014', '4 Mar, 2009', '15 Apr, 2010', '30 Apr, 2012', '16 Feb, 2010', '14 Feb, 2020', '24 Oct, 2011', '4 Oct, 2016', '26 Oct, 2006', '7 Oct, 2014', '7 Sep, 2012', '26 May, 2015', '24 Apr, 2012', '10 Sep, 2004', '1 Mar, 2012', '6 May, 2014', '23 Jul, 2013', '19 Dec, 2011', '1 Jun, 2012', '4 Oct, 2010', '4 Mar, 2020', '17 Dec, 2012', '1 Jun, 1991', '11 Sep, 2012', '31 May, 2022', '18 May, 2010', '24 Nov, 1988', '12 Dec, 2009', '12 Apr, 2020', '12 Sep, 2013', '30 Aug, 2010', '21 May, 2015', '10 Dec, 2013', '17 Jul, 2009', '22 Jun, 2010', '20 Jul, 2010', '18 Jul, 2008', '7 Jun, 2013', '14 Aug, 2012', '4 Mar, 2014', '22 May, 2015', '16 Oct, 2006', '14 Nov, 2007', '5 Dec, 2012', '2 Jun, 2015', '27 Aug, 2008', '9 May, 2008', '16 May, 2011', '26 Oct, 2015', '29 Aug, 2008'])


La cantidad de datos nulos en 'release_date' son: 57


Lo mismo hacemos con las columnas "genres" y "categories".

In [9]:
cant_nulos_genres = data_cpy.isnull().sum()["genres"]
print ("La cantidad de datos nulos en 'genres' son: " + str(cant_nulos_genres))

arr_genres = np.array(['RPG', 'Adventure', 'Action, Adventure, RPG', 'Action, Adventure', 'RPG', 'Action, RPG, Adventure', 'Strategy', 'Strategy', 'Strategy', 'Adventure, Indie', 'Strategy', 'Adventure', 'Simulation', 'RPG, Strategy', 'Indie, Simulation', 'Adventure, Simulation, Strategy', 'Casual, Indie, RPG', 'Cusual', 'Strategy', 'Action, Adventure, RPG', 'Action, Adventure', 'Action', 'Action, RPG', 'Action', 'Cusual, Indie, Strategy', 'Action', 'Action, RPG', 'Casual, Strategy', 'Casual', 'Action, Adventure', 'Casual', 'Simulation', 'Strategy', 'Casual, Indie', 'Casual', 'Adventure', 'Action', 'Strategy', 'Strategy', 'Action, Adventure', 'Action', 'Action, Indie, Simulation, Strategy', 'Adventure', 'Action, Indie, Simulation, Strategy', 'Casual', 'Casual', 'Action', 'Action, Simulation', 'Action', 'Casual', 'Action, Indie', 'Casual', 'Action', 'Action', 'Casual', 'Action', 'Strategy', 'Action, Adventure', 'Action', 'Strategy', 'Casual', 'Action', 'Action, Simulation, Strategy', 'Action, RPG', 'Casual', 'Action, Adventure', 'RPG', 'Strategy', 'Action', 'Casual', 'Action, Adventure', 'Adventure', 'Simulation', 'Adventure, Indie', 'Casual, Simulation', 'RPG', 'Action, Indie', 'Adventure', 'Strategy', 'Violent, Action', 'Casual', 'Action, Violent', 'Action', 'Strategy', 'Action, Simulation', 'Indie', 'Casual, Indie'])

arr_genres.shape

La cantidad de datos nulos en 'genres' son: 87


(87,)

In [10]:
data_cpy.loc[data_cpy['release_date'].isnull(), 'release_date'] = arr_release_date

cant_nulos_release_date = data_cpy.isnull().sum()["release_date"]
print ("La cantidad de datos nulos en 'release_date' son: " + str(cant_nulos_release_date))

La cantidad de datos nulos en 'release_date' son: 0


In [11]:
data_cpy.loc[data_cpy['genres'].isnull(), 'genres'] = arr_genres

cant_nulos_genres = data_cpy.isnull().sum()["genres"]
print ("La cantidad de datos nulos en 'genres' son: " + str(cant_nulos_genres))

La cantidad de datos nulos en 'genres' son: 0


In [12]:
cant_nulos_categories = data_cpy.isnull().sum()["categories"]
print ("La cantidad de datos nulos en 'categories' son: " + str(cant_nulos_categories))

arr_categories = np.array(['Single-player, Downloadable Content, Steam Achievements, Steam Cloud, Remote Play on Phone, Remote Play on Tablet, Family Sharing', 'Single-player, Downloadable Content, Steam Achievements, Steam Trading Cards, In-App Purchases, Steam Cloud, Family Sharing', 'Single-player, Downloadable Content, Steam Achievements, Steam Trading Cards, Steam Cloud, Steam Leaderboards, Remote Play on Phone, Remote Play on Tablet, Remote Play on TV, Family Sharing', 'Single-player, Downloadable Content, Steam Achievements, Steam Cloud, Remote Play on Phone, Remote Play on Tablet, Remote Play on TV, Family Sharing', 'Single-player, Steam Trading Cards, Steam Cloud, Family Sharing', 'Single-player, Downloadable Content, Steam Achievements, Steam Trading Cards, Steam Cloud, Stats, Steam Leaderboards, Family Sharing', 'Single-player, Downloadable Content, Steam Achievements, Steam Trading Cards, Steam Cloud, Stats, Steam Leaderboards, Family Sharing', 'Single-player, Steam Achievements, Steam Trading Cards, Steam Cloud, Stats, Steam Leaderboards, Family Sharing', 'Single-player, Steam Cloud, Family Sharing', 'Single-player, Downloadable Content, Steam Achievements, Steam Trading Cards, Steam Cloud, Remote Play on TV, Family Sharing', 'Single-player, Downloadable Content, Steam Achievements, Steam Trading Cards, Captions available, In-App Purchases, Steam Cloud, Steam Leaderboards, Remote Play on Phone, Remote Play on Tablet, Remote Play on TV, Family Sharing', 'Single-player', 'Single-player, Online PvP, Downloadable Content, Steam Achievements, Steam Trading Cards, In-App Purchases, Steam Cloud, HDR available, Family Sharing', 'Single-player, Online PvP, Online Co-op, Downloadable Content, Steam Achievements, Steam Trading Cards, Steam Workshop, Steam Cloud, Valve Anti-Cheat enabled, Includes level editor, Remote Play on Tablet, Remote Play on TV, Family Sharing', 'Single-player, Online PvP, Shared/Split Screen PvP, Downloadable Content, Steam Achievements, Steam Trading Cards, In-App Purchases, Steam Cloud, Remote Play on Phone, Remote Play on Tablet, Remote Play on TV, Remote Play Together, HDR available, Family Sharing', 'Single-player, Downloadable Content, Steam Achievements, Steam Trading Cards, Steam Cloud, Remote Play on Phone, Remote Play on Tablet, Remote Play on TV, Family Sharing', 'Single-player, Downloadable Content, Steam Achievements, Steam Cloud, Steam Leaderboards, Remote Play on Tablet, Remote Play on TV, Family Sharing', 'Single-player, Steam Achievements, Family Sharing', 'Single-player, Online PvP, Downloadable Content, Steam Achievements, Steam Trading Cards, In-App Purchases, Steam Cloud, HDR available, Family Sharing', 'Single-player, Downloadable Content, Steam Achievements, Steam Cloud, Steam Leaderboards, Family Sharing', 'Single-player, Online PvP, Online Co-op, Downloadable Content, Steam Achievements, Steam Trading Cards, Steam Workshop, In-App Purchases, Steam Cloud, Stats, Steam Leaderboards, Includes level editor, Family Sharing', 'Single-player, Online PvP, Online Co-op, Downloadable Content, Steam Achievements, Steam Trading Cards, Steam Workshop, In-App Purchases, Steam Cloud, Stats, Steam Leaderboards, Includes level editor, Family Sharing', 'Single-player, Steam Achievements, Steam Cloud, Steam Leaderboards, Family Sharing', 'Single-player, Shared/Split Screen PvP, Downloadable Content, Steam Achievements, Steam Trading Cards, Steam Cloud, Steam Leaderboards, Remote Play on Phone, Remote Play on Tablet, Remote Play on TV, Remote Play Together, Family Sharing', 'Single-player, Steam Achievements, Family Sharing', 'Single-player, Downloadable Content, Steam Achievements, Steam Trading Cards, Captions available, Steam Cloud, Remote Play on TV, Family Sharing', 'Steam Achievements, Steam Workshop, Steam Cloud, Stats, Includes level editor, Family Sharing', 'Single-player, Downloadable Content', 'Single-player, Steam Achievements, Steam Trading Cards, Steam Cloud, Remote Play on Tablet, Remote Play on TV, Family Sharing', 'Single-player, Online PvP, Online Co-op, Downloadable Content, Captions available, Includes level editor, Family Sharing', 'Single-player, Steam Achievements, Steam Cloud, Family Sharing', 'Online PvP, Downloadable Content', 'Single-player', 'Online PvP', 'Single-player, Steam Cloud, Remote Play on Phone, Remote Play on Tablet, Remote Play Together, Family Sharing', 'Single-player, Downloadable Content, Steam Achievements, Steam Trading Cards, Steam Cloud, Remote Play on TV, Family Sharing', 'Single-player', 'Online PvP', 'Single-player, Online PvP, Online Co-op, Cross-Platform Multiplayer, Downloadable Content, Steam Achievements, Steam Trading Cards, In-App Purchases, Steam Cloud, Remote Play on TV, Family Sharing', 'Online PvP', 'Single-player, Online PvP, Downloadable Content, Steam Achievements, Steam Cloud, Family Sharing', 'Online PvP', 'Online PvP', 'Single-player, Online PvP, Downloadable Content, Steam Achievements, Steam Cloud, Family Sharing', 'Online PvP, Online Co-op'])

arr_categories.shape

La cantidad de datos nulos en 'categories' son: 45


(45,)

In [13]:
data_cpy.loc[data_cpy['categories'].isnull(), 'categories'] = arr_categories

cant_nulos_categories = data_cpy.isnull().sum()["categories"]
print ("La cantidad de datos nulos en 'categories' son: " + str(cant_nulos_categories))

La cantidad de datos nulos en 'categories' son: 0


---
A continuación se eliminan las filas que tengan valores nulos en release_date, genre, categories

In [14]:
cant_nulos_release_date = data_cpy.isnull().sum()["release_date"]
print ("La cantidad de datos nulos en 'release_date' son: " + str(cant_nulos_release_date))

La cantidad de datos nulos en 'release_date' son: 0


In [15]:
data_cpy = data_cpy.dropna(subset=['release_date'])

cant_nulos_release_date_after = data_cpy['release_date'].isnull().sum()
print ("La cantidad de datos nulos en 'release_date' son: " + str(cant_nulos_release_date_after))

La cantidad de datos nulos en 'release_date' son: 0


In [16]:
cant_nulos_genres = data_cpy.isnull().sum()["genres"]
print ("La cantidad de datos nulos en 'genres' son: " + str(cant_nulos_genres))

La cantidad de datos nulos en 'genres' son: 0


In [17]:
data_cpy = data_cpy.dropna(subset=['genres'])

cant_nulos_genres_after = data_cpy['genres'].isnull().sum()
print ("La cantidad de datos nulos en 'genres' son: " + str(cant_nulos_genres_after))

La cantidad de datos nulos en 'genres' son: 0


In [18]:
cant_nulos_categories = data_cpy.isnull().sum()["categories"]
print ("La cantidad de datos nulos en 'categories' son: " + str(cant_nulos_categories))

La cantidad de datos nulos en 'categories' son: 0


In [19]:
data_cpy = data_cpy.dropna(subset=['categories'])

cant_nulos_categories_after = data_cpy['categories'].isnull().sum()
print ("La cantidad de datos nulos en 'categories' son: " + str(cant_nulos_categories_after))

La cantidad de datos nulos en 'categories' son: 0




---


Para la columna de "developer" y "publisher" agregamos el valor "Unknown" para los valores nulos

In [20]:
data_cpy.loc[data_cpy["developer"].isnull(), "developer"] = "Unknown"

cant_nulos_developer = data_cpy.isnull().sum()["developer"]
print ("La cantidad de datos nulos en 'developer' son: " + str(cant_nulos_developer))

data_cpy.loc[data_cpy["publisher"].isnull(), "publisher"] = "Unknown"

cant_nulos_publisher = data_cpy.isnull().sum()["publisher"]
print ("La cantidad de datos nulos en 'publisher' son: " + str(cant_nulos_publisher))

La cantidad de datos nulos en 'developer' son: 0
La cantidad de datos nulos en 'publisher' son: 0


---
Para la columna del precio de descuento asumimos que los articulos que no tienen precio con el descuento son gratis

In [21]:
data_cpy.loc[data_cpy["discounted_price"].isnull(), "discounted_price"] = "Free"

Si el discounted price es Free, el original price y el discount_percentage son 0.

In [22]:
data_cpy.loc[(data_cpy["discounted_price"] == "Free") & (data_cpy["original_price"].isnull()), "original_price"] = 0

In [23]:
data_cpy.loc[(data_cpy["discounted_price"] == "Free") & (data_cpy["discount_percentage"].isnull()), "discount_percentage"] = 0

Si el 'discounted_price' no es Free si ambos 'discounted_price' y 'original_price' son 0, asumimos que no se aplicaron descuentos y que el precio original es el mismo que el 'discounted_price'

In [24]:
data_cpy.loc[(data_cpy["discounted_price"] != "Free") & (data_cpy["discount_percentage"].isnull()) & (data_cpy["original_price"].isnull()), "discount_percentage"] = '0'
data_cpy.loc[(data_cpy["discounted_price"] != "Free") & (data_cpy["discount_percentage"] == '0') & (data_cpy["original_price"].isnull()), "original_price"] = data_cpy["discounted_price"]


---
Para la columna "content_descriptor" si es null asumimos que no tiene contenido relevante que informar al jugador por lo agregamos el valor "Not relevant content descriptor"

In [25]:
data_cpy.loc[data_cpy["content_descriptor"].isnull(), "content_descriptor"] = "Not relevant content descriptor"

Para la columna "about_description" si el valor es null se reemplaza por "Unknown"



In [26]:
data_cpy.loc[data_cpy["about_description"].isnull(), "about_description"] = "Unknown"

Para las columnas "overall_review" y "recent_review" si el valor es null se reemplaza por "No reviews"


In [27]:
data_cpy.loc[data_cpy["overall_review"].isnull(), "overall_review"] = "No reviews"
data_cpy.loc[data_cpy["recent_review"].isnull(), "recent_review"] = "No reviews"

In [28]:
print("La cantidad de datos nulos en 'overall_review' son: " + str(data_cpy["overall_review"].isnull().sum()))
print("La cantidad de datos nulos en 'recent_review' son: " + str(data_cpy["recent_review"].isnull().sum()))

La cantidad de datos nulos en 'overall_review' son: 0
La cantidad de datos nulos en 'recent_review' son: 0


Para las columnas "overall_review_%" y "recent_review_%" si el valor es null se reemplaza por "0%"



In [29]:
data_cpy.loc[data_cpy["overall_review_%"].isnull(), "overall_review_%"] = 0
data_cpy.loc[data_cpy["recent_review_%"].isnull(), "recent_review_%"] = 0

In [30]:
print("La cantidad de datos nulos en 'overall_review_%' son: " + str(data_cpy["overall_review_%"].isnull().sum()))
print("La cantidad de datos nulos en 'recent_review_%' son: " + str(data_cpy["recent_review_%"].isnull().sum()))

La cantidad de datos nulos en 'overall_review_%' son: 0
La cantidad de datos nulos en 'recent_review_%' son: 0


Para las columnas "overall_review__count" y "recent_review__count" si el valor es null se reemplaza por '0'

In [31]:
data_cpy.loc[data_cpy["overall_review_count"].isnull(), "overall_review_count"] = 0
data_cpy.loc[data_cpy["recent_review_count"].isnull(), "recent_review_count"] = 0

In [32]:
print("La cantidad de datos nulos en 'overall_review_count' son: " + str(data_cpy["overall_review_count"].isnull().sum()))
print("La cantidad de datos nulos en 'recent_review_count' son: " + str(data_cpy["recent_review_count"].isnull().sum()))

La cantidad de datos nulos en 'overall_review_count' son: 0
La cantidad de datos nulos en 'recent_review_count' son: 0




---

Podemos verificar a continuación que cubrimos todos los valores nulos.

In [33]:
data_cpy.isnull().sum()

app_id                  0
title                   0
release_date            0
genres                  0
categories              0
developer               0
publisher               0
original_price          0
discount_percentage     0
discounted_price        0
dlc_available           0
age_rating              0
content_descriptor      0
about_description       0
win_support             0
mac_support             0
linux_support           0
awards                  0
overall_review          0
overall_review_%        0
overall_review_count    0
recent_review           0
recent_review_%         0
recent_review_count     0
dtype: int64

## Conversion de tipos de Datos


Convertimos la columna 'title' de tipo object a 'str'

In [34]:
tipo_de_dato_title = data_cpy['title'].dtype

In [35]:
data_cpy['title'] = data_cpy['title'].astype('str')
data_cpy['title'].dtype

dtype('O')

---
Convertimos 'release_date' de tipo object a 'datetime'

In [36]:
data_cpy['release_date'].dtype

dtype('O')

Analizamos como es el formato para ver si pueden ser convertidos a datetime

In [37]:
data_cpy['release_date'].head(5)

0    21 Aug, 2012
1     9 Jul, 2013
2    16 May, 2024
3    24 Feb, 2022
4     1 Oct, 2019
Name: release_date, dtype: object

Vemos que parecen tener el siguiente formato '%d %b, %Y'

In [38]:
data_cpy['release_date'] = data_cpy['release_date'].apply(lambda x: pd.to_datetime(x, format='%d %b, %Y',errors='ignore'))


Intentamos convertir con ese formato a datetime y vemos cuantos y cuales son los valores que no pudieron ser convertidos.

In [39]:
non_datetime_values = data_cpy.loc[~data_cpy['release_date'].apply(lambda x: isinstance(x, datetime.datetime)), 'release_date']

In [40]:
non_datetime_values.shape

(64,)

In [41]:
non_datetime_values.head()

274     Apr 2019
425     Mar 2021
991     Nov 2020
1028    Nov 2014
1451    Dec 2016
Name: release_date, dtype: object

In [42]:
non_datetime_values.unique()

array(['Apr 2019', 'Mar 2021', 'Nov 2020', 'Nov 2014', 'Dec 2016',
       'Jan 2021', 'Apr 2022', 'Apr 2020', 'Aug 2015', 'Oct 2010',
       'Jan 2019', 'Aug 2017', 'Dec 2012', 'Mar 2019', 'Jul 2010',
       'May 2015', 'Jul 2016', 'Jan 2017', 'May 2014', 'Jun 2009',
       'Dec 2019', 'Feb 2020', 'Mar 2010', 'Apr 2010', 'Jan 2022',
       'Oct 2020', 'Apr 2015', 'Jun 2021', 'Jan 2018', 'Aug 2013',
       'Dec 2020', 'Dec 2015', 'May 2019', 'Jul 2018', 'Aug 2021',
       'May 2017', 'Feb 2022', 'Jul 2017', 'Jun 2015', 'Oct 2018',
       'Oct 2009', 'Sep 2014', 'Sep 2016', 'Jul 2022', 'Sep 2019',
       'Mar 2016', 'Apr 2016', 'Coming soon', 'Sep 2020', 'Jan 2010',
       'To be announced', 'Nov 2021'], dtype=object)

Vemos que parece que los datos que no se pudieron convertir son del tipo '%b %Y' menos dos que son 'Coming soon' y 'To be announced'. Decidimos descartar esas filas.

In [43]:
data_cpy = data_cpy[~data_cpy["release_date"].isin(["Coming soon", "To be announced"])]

Ajustamos formato de fecha para que sea '%d %b, %Y'

In [44]:
def format_date(date):
    try:
        month, year = date.split()
        date = f"{month}, {year}"
        return pd.to_datetime('1 ' + date, format='%d %b, %Y')
    except:
        return date
data_cpy['release_date'] = data_cpy['release_date'].apply(format_date)

Vemos que se convirtieron todos los datos a datetime con **formato '%d %b, %Y'**

In [45]:
non_datetime_values = data_cpy.loc[~data_cpy['release_date'].apply(lambda x: isinstance(x, datetime.datetime)), 'release_date']

In [46]:
non_datetime_values.shape

(0,)

In [47]:
data_cpy['release_date'].dtype

dtype('<M8[ns]')


---




Convertimos los porcentajes de 'recent_review_%' y 'overall_review_%' a enteros.

In [48]:
data_cpy['recent_review_%'] = data_cpy['recent_review_%'].astype('Int64')
data_cpy['overall_review_%'] = data_cpy['overall_review_%'].astype('Int64')

Convertimos 'recent_review_count' y 'overall_review_count' a enteros.

In [49]:
data_cpy['recent_review_count'] = data_cpy['recent_review_count'].astype('Int64')
data_cpy['overall_review_count'] = data_cpy['overall_review_count'].astype('Int64')

---
Los precios los convertimos a numeros flotantes para poder trabajarlos de mejor manera. A su vez agregamos la moneda al nombre de la columna para poder tener la referencia correspondiente.

In [50]:
data_cpy['discounted_price'].unique()

array(['Free', '₹3,999.00', '₹3,599.00', ..., '₹461.00', '₹1,075.00',
       '₹150,000.00'], dtype=object)

In [51]:
data_cpy['original_price'] = data_cpy['original_price'].replace('Free', '₹0.00')
data_cpy['original_price'] = data_cpy['original_price'].replace('[₹,]', '', regex=True).astype(float)
data_cpy.rename(columns={'original_price': 'original_price_INR'}, inplace=True)

data_cpy['discounted_price'] = data_cpy['discounted_price'].replace('Free', '₹0.00')
data_cpy['discounted_price'] = data_cpy['discounted_price'].replace('[₹,]', '', regex=True).astype(float)
data_cpy.rename(columns={'discounted_price': 'discounted_price_INR'}, inplace=True)

---
Así llegamos a los siguientes tipos de datos para toda la tabla

In [52]:
data_cpy.dtypes

app_id                           int64
title                           object
release_date            datetime64[ns]
genres                          object
categories                      object
developer                       object
publisher                       object
original_price_INR             float64
discount_percentage             object
discounted_price_INR           float64
dlc_available                    int64
age_rating                       int64
content_descriptor              object
about_description               object
win_support                       bool
mac_support                       bool
linux_support                     bool
awards                           int64
overall_review                  object
overall_review_%                 Int64
overall_review_count             Int64
recent_review                   object
recent_review_%                  Int64
recent_review_count              Int64
dtype: object

## Validación de rangos de valores

Se procede a borrar las filas que contienen valores con porcentajes invalidos (porcentajes menores que 0 o mayores a 100)

In [53]:
print("Cantidad de filas antes de filtrar:", data_cpy.shape[0])

data_cpy = data_cpy[(data_cpy['overall_review_%'] >= 0) & (data_cpy['overall_review_%'] <= 100)]
data_cpy = data_cpy[(data_cpy['recent_review_%'] >= 0) & (data_cpy['recent_review_%'] <= 100)]

print("Cantidad de filas después de filtrar:", data_cpy.shape[0])



Cantidad de filas antes de filtrar: 42495
Cantidad de filas después de filtrar: 42495



Se procede a borrar las filas que contienen valores fuera de rango (por ejemplo, precios con valores negativos)



In [54]:
print("Cantidad de filas antes de filtrar:", data_cpy.shape[0])

data_cpy = data_cpy[(data_cpy['app_id'] >= 0)]
data_cpy = data_cpy[(data_cpy['original_price_INR'] >= 0)]
data_cpy = data_cpy[(data_cpy['discounted_price_INR'] >= 0)]
data_cpy = data_cpy[(data_cpy['dlc_available'] >= 0)]
data_cpy = data_cpy[(data_cpy['age_rating'] >= 0)]
data_cpy = data_cpy[(data_cpy['awards'] >=  0)]
data_cpy = data_cpy[(data_cpy['overall_review_count'] >= 0)]
data_cpy = data_cpy[(data_cpy['recent_review_count'] >= 0)]

print("Cantidad de filas después de filtrar:", data_cpy.shape[0])

Cantidad de filas antes de filtrar: 42495
Cantidad de filas después de filtrar: 42495


Cambiamos el nombre de las columnas que incluyen '%' para evitar problemas con sqlite.

In [55]:
data_cpy = data_cpy.rename(columns={'overall_review_%': 'overall_review_percentage', 'recent_review_%': 'recent_review_percentage'})

# Carga de datos limpios en Base de Datos

In [56]:
import sqlite3

# Conectar a la base de datos SQLite
conn = sqlite3.connect('tpp_bdd.sqlite')
cur = conn.cursor()

# Crear la tabla principal 'games'
cur.execute('''
CREATE TABLE IF NOT EXISTS games (
    app_id INTEGER PRIMARY KEY,
    title TEXT,
    release_date TEXT,
    developer TEXT,
    publisher TEXT,
    original_price_INR REAL,
    discount_percentage REAL,
    discounted_price_INR REAL,
    dlc_available INTEGER,
    age_rating INTEGER,
    content_descriptor TEXT,
    about_description TEXT,
    win_support BOOLEAN,
    mac_support BOOLEAN,
    linux_support BOOLEAN,
    awards INTEGER,
    overall_review TEXT,
    overall_review_percentage INTEGER,
    overall_review_count REAL,
    recent_review TEXT,
    recent_review_percentage INTEGER,
    recent_review_count REAL
)
''')

# Crear la tabla 'game_genres'
cur.execute('''
CREATE TABLE IF NOT EXISTS game_genres (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    app_id INTEGER,
    genre TEXT,
    FOREIGN KEY (app_id) REFERENCES games (app_id)
)
''')

# Crear la tabla 'game_categories'
cur.execute('''
CREATE TABLE IF NOT EXISTS game_categories (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    app_id INTEGER,
    category TEXT,
    FOREIGN KEY (app_id) REFERENCES games (app_id)
)
''')

# Confirmar los cambios
conn.commit()


# Insertar los datos del DataFrame 'df' en la tabla 'games'
df_games = data_cpy[['app_id', 'title', 'release_date', 'developer', 'publisher', 'original_price_INR',
               'discount_percentage', 'discounted_price_INR', 'dlc_available', 'age_rating',
               'content_descriptor', 'about_description', 'win_support', 'mac_support',
               'linux_support', 'awards', 'overall_review', 'overall_review_percentage', 'overall_review_count',
               'recent_review', 'recent_review_percentage', 'recent_review_count']]

df_games.to_sql('games', conn, if_exists='append', index=False)


# Insertar datos en la tabla 'game_genres'
genres_data = []
for index, row in data_cpy.iterrows():
    genres = str(row['genres']).split(',')
    for genre in genres:
        genres_data.append((row['app_id'], genre.strip()))

cur.executemany('INSERT INTO game_genres (app_id, genre) VALUES (?, ?)', genres_data)

# Insertar datos en la tabla 'game_categories'
categories_data = []
for index, row in data_cpy.iterrows():
    categories = str(row['categories']).split(',')
    for category in categories:
        categories_data.append((row['app_id'], category.strip()))

cur.executemany('INSERT INTO game_categories (app_id, category) VALUES (?, ?)', categories_data)

conn.commit()
conn.close()